# Natural language engineering 2: Assignment

## Classification of Twitter posts regarding the Ukrainian War by emotion

A project by **Johannes Wittmann** and **Quirin Wittmann**.

In [1]:
from pprint import pprint

### Data crawling

### Data preprocessing

### Model finetuning

Neccesary dependencys for finetuning with Huggingface.

In [2]:
!pip install datasets transformers
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 7.3 MB/s 
     |████████████████████████████████| 4.4 MB 59.7 MB/s 
     |████████████████████████████████| 1.1 MB 41.0 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 140 kB 52.9 MB/s 
     |████████████████████████████████| 212 kB 56.2 MB/s 
     |████████████████████████████████| 596 kB 59.8 MB/s 
     |████████████████████████████████| 127 kB 52.6 MB/s 
     |████████████████████████████████| 6.6 MB 37.7 MB/s 
     |████████████████████████████████| 271 kB 14.9 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 144 kB 77.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing i

Finetuning imports:


In [3]:
from datasets import load_dataset
from transformers import pipeline
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

Load Twitter dataset for emotions: <br>
[https://huggingface.co/datasets/tweet_eval/viewer/emotion/train](https://huggingface.co/datasets/tweet_eval/viewer/emotion/train)

In [4]:
emotion_dataset = load_dataset("tweet_eval", "emotion")
print(emotion_dataset["train"][0])

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3257 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1421 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/374 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emotion/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': "“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry", 'label': 2}


Embedd tweets:

In [5]:
#checkpoint = "bhadresh-savani/distilbert-base-uncased-emotion"
#checkpoint = "bert-base-uncased"
checkpoint = "bert-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Tokenisation function defining:

In [6]:
def tokenize_example(example):
  return tokenizer(example["text"], truncation=True)

Tokenize dataset with function:

In [7]:
emotion_dataset_tokenized = emotion_dataset.map(tokenize_example, batched=True)

Parameter 'function'=<function tokenize_example at 0x7fbe79d26710> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Defining the dataset padder:

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Classification Head:

In [9]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4, ignore_mismatched_sizes=True)

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Define the evaluation function.
Huggingface reccomends accuracy, recall, precision and F1 for text classification. <br>
[Metrics for Text Classification](https://huggingface.co/tasks/text-classification)

In [10]:
def evaluate(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="macro")
  accuracy = accuracy_score(labels, preds)
  precision = precision_score(labels, preds, average="macro")
  recall = recall_score(labels, preds, average="macro")
  return {"precision": precision, "recall": recall, "accuracy": accuracy, "f1": f1}

Defining the trainingparameters:

In [11]:
batch_size = 8
logging_steps = len(emotion_dataset_tokenized["train"])
model_name = f"{checkpoint}-finetuned-tweet_emotion"
training_args = TrainingArguments(output_dir = model_name, 
                                  num_train_epochs = 9, 
                                  learning_rate = 2e-5, 
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  evaluation_strategy="epoch",
                                  disable_tqdm = False, 
                                  logging_steps = logging_steps, 
                                  log_level="error")

Creating the trainer:

In [12]:
print(len(emotion_dataset_tokenized["train"]))
print(len(emotion_dataset_tokenized["test"]))
print(len(emotion_dataset_tokenized["validation"]))
trainer = Trainer(
    model = model,
    args = training_args,
    compute_metrics = evaluate,
    train_dataset=emotion_dataset_tokenized["train"], 
    eval_dataset=emotion_dataset_tokenized["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

3257
1421
374


Training:

In [13]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Precision,Recall,Accuracy,F1
1,No log,0.597881,0.799816,0.713903,0.787474,0.741310
2,No log,0.602484,0.799353,0.770784,0.818438,0.782651
3,No log,0.838110,0.785634,0.795669,0.813512,0.789259
4,No log,0.979788,0.801352,0.805928,0.829697,0.803396
5,No log,1.143337,0.801763,0.796774,0.821956,0.797631
6,No log,1.194743,0.796806,0.800888,0.824068,0.798127
7,No log,1.325963,0.805203,0.796438,0.826179,0.800087
8,0.216600,1.345655,0.800253,0.788930,0.821956,0.793950
9,0.216600,1.398085,0.800391,0.786920,0.821253,0.792348


TrainOutput(global_step=3672, training_loss=0.1925704336374154, metrics={'train_runtime': 1414.581, 'train_samples_per_second': 20.722, 'train_steps_per_second': 2.596, 'total_flos': 2081555573312448.0, 'train_loss': 0.1925704336374154, 'epoch': 9.0})

In [14]:
trainer.save_model("./outputmodel")

### Classification

In [15]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from transformers import pipeline
from transformers import AutoModelForSequenceClassification

In [17]:
import os
os.listdir("/content/outputmodel/")

['pytorch_model.bin',
 'vocab.txt',
 'tokenizer.json',
 'special_tokens_map.json',
 'config.json',
 'tokenizer_config.json',
 'training_args.bin']

In [18]:
checkpoint = "/content/outputmodel/"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=4)
emotion_pipe = pipeline(task="sentiment-analysis", model=model, tokenizer=tokenizer)

print(model.config.id2label)

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2', 3: 'LABEL_3'}


In [22]:
# TEST DER PIPELINE
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
    "I love this so much",
    "I am looking forward to it",
    "We must stay optimistic",
    "Why did this happen?",
    "Why did you do this to me?",
    "Why didn't you clean this mess?",
    "I am sorry",
    "I am so excited",
]
#0: anger
#1: joy
#2: optimism
#3: sadness
pprint(emotion_pipe(raw_inputs))

[{'label': 'LABEL_1', 'score': 0.9997829794883728},
 {'label': 'LABEL_0', 'score': 0.9999632835388184},
 {'label': 'LABEL_1', 'score': 0.9999147653579712},
 {'label': 'LABEL_1', 'score': 0.9998960494995117},
 {'label': 'LABEL_2', 'score': 0.999860405921936},
 {'label': 'LABEL_0', 'score': 0.9999475479125977},
 {'label': 'LABEL_0', 'score': 0.9999574422836304},
 {'label': 'LABEL_0', 'score': 0.9999207258224487},
 {'label': 'LABEL_3', 'score': 0.999948263168335},
 {'label': 'LABEL_1', 'score': 0.999942421913147}]


In [20]:
#predictions = emotion_pipe( HIER GECRAWLTE TWEET-LISTE)

In [21]:
predicted_labels = []
for prediction in predictions:
  predicted_labels.append(prediction['label'].lower())

NameError: ignored

### Data analysis and postprocessing

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fa2802eb-518b-4254-91a1-6660d89efb87' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>